In [2]:
import pandas as pd
import numpy as np
import pandasql as ps
import matplotlib.pyplot as plt


from matplotlib.ticker import MultipleLocator
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_openml
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

np.random.seed(0)

In [3]:
#load dataframe from csv
merged_sales = pd.read_csv("data/zip/merged_sales_stats.csv", delimiter='	')

#print dataframe shape
shape = merged_sales.shape
print('\nDataFrame Shape :', shape)

print(merged_sales.columns)


DataFrame Shape : (137166, 23)
Index(['Unnamed: 0', 'Zip Code', 'Month of Period End', 'Median Sale Price',
       'Median Sale Price MoM ', 'Median Sale Price YoY ', 'Homes Sold',
       'Homes Sold MoM ', 'Homes Sold YoY ', 'New Listings',
       'New Listings MoM ', 'New Listings YoY ', 'Inventory', 'Inventory MoM ',
       ' Inventory YoY ', 'Days on Market', 'Days on Market MoM',
       'Days on Market YoY', 'Average Sale To List',
       'Average Sale To List MoM ', 'Average Sale To List YoY ',
       'record_month', 'PSSF'],
      dtype='object')


The merged_sales_statistics file has all the statistics as columns and zip codes and dates as the rows. I tried to transpose these but it got weird. So let's get some stuff from the merged_sales:

In [4]:


#load dataframe from csv
merged_bedroom_data = pd.read_csv("data/zip/merged_bedrooms.csv", delimiter='	')
print(merged_bedroom_data.shape)
print(merged_bedroom_data.columns)


(1688, 682)
Index(['Unnamed: 0', 'Zip Code', 'June 2009 1bd', 'July 2009 1bd',
       'August 2009 1bd', 'September 2009 1bd', 'October 2009 1bd',
       'November 2009 1bd', 'December 2009 1bd', 'January 2010 1bd',
       ...
       'December 2019 5bd', 'January 2020 5bd', 'February 2020 5bd',
       'March 2020 5bd', 'April 2020 5bd', 'May 2020 5bd', 'June 2020 5bd',
       'July 2020 5bd', 'August 2020 5bd', 'September 2020 5bd'],
      dtype='object', length=682)


In [5]:
''' 
now for our demographic info, it's from 2019 so not super up to date, but pretty close and there's a lot of it
'''
demographics = pd.read_csv("data/zip/ca_demographics.csv", delimiter='	')
zip_to_county = pd.read_csv("data/zip/zip_to_county.csv", delimiter='	')

print(demographics.shape) # a lot of blank columns


print(demographics["SE_A18003_001"])

(41, 207)
0     29.100000
1     31.900000
2     30.700001
3     37.599998
4     33.000000
5     36.099998
6     30.799999
7     33.700001
8     29.500000
9     29.100000
10    33.299999
11    30.299999
12    32.099998
13    31.799999
14    27.299999
15    32.799999
16    29.900000
17    42.599998
18    32.099998
19    30.799999
20    33.200001
21    32.400002
22    33.000000
23    32.700001
24    22.299999
25    31.500000
26    33.099998
27    27.600000
28    33.000000
29    27.600000
30    32.200001
31    27.200001
32    30.600000
33    31.500000
34    31.700001
35    33.500000
36    26.700001
37    29.400000
38    33.200001
39    31.000000
40    30.600000
Name: SE_A18003_001, dtype: float64


Most of the column names are gibberish, so we'll need to use the [ca_demographics_key.txt](data/zip/ca_demographics_key.txt) to make sense of them.

In [6]:
zipdemo = pd.read_csv("data/zip/ca_zip_demograhics.csv", delimiter='	')
print(zipdemo.shape)

(1763, 217)


There's a lot to look at in that dataset, but certainly all the key indicators about the zip should give us at least some picture of what the housing market there might look like.

##### Condensing Sale Data to Year

In [7]:
#remove month from "Month of Period End"
only_year_query = """SELECT "Zip Code" as zip_code, 
                    SUBSTR("Month of Period End" ,-4) as year,
                    "Median Sale Price" as med_sale_price,
                    "Homes Sold" as home_sold,
                    "New Listings" as new_listings,
                    "Inventory" as inventory,
                    "Days on Market" as days_on_market,
                    "Average Sale To List" as avg_sale_to_list,
                    PSSF as ppsf
                    FROM merged_sales"""
only_year = ps.sqldf(only_year_query, locals())

#group by year
cond_year_query = """SELECT zip_code, 
                            year,
                            AVG(med_sale_price) as med_sale_price,
                            SUM(home_sold) as homes_sold,
                            SUM(new_listings) as new_listings,
                            AVG(inventory) as inventory,
                            AVG(days_on_market) as days_on_market,
                            AVG(avg_sale_to_list) as avg_sale_to_list,
                            AVG(ppsf) as ppsf
                    FROM only_year
                    GROUP BY year, zip_code"""
cond_year = ps.sqldf(cond_year_query, locals())

#filtering to 2018
year_18_query = """SELECT *
                    FROM cond_year
                    WHERE year = '2018' """
year_18 = ps.sqldf(year_18_query, locals())

### 2018 Sales and Demographic data

In [8]:
#zip code = Geo_ZCTA5

sales_demo_query = """SELECT year_18.*, zipdemo.*
                    FROM year_18
                    INNER JOIN zipdemo on zip_code = Geo_ZCTA5
                    """
sales_demo = ps.sqldf(sales_demo_query, locals())
sales_demo

,zip_code,year,med_sale_price,homes_sold,new_listings,inventory,days_on_market,avg_sale_to_list,ppsf,Geo_FIPS,...,SE_A08002B_005,SE_A08002B_006,SE_A10066_001,SE_A10066_002,SE_A10066_003,SE_A10066_004,SE_A10066_005,SE_A10066_006,SE_A10066_007,SE_A10066_008
0,90001,2018,4.228333e+05,511,678.0,44.333333,43.166667,100.125000,322.500000,90090001,...,144,118,13815,1834,2083,2594,2513,2045,1330,1416
1,90002,2018,3.825000e+05,964,1178.0,74.666667,39.916667,99.916667,323.750000,90090002,...,45,124,12706,2096,2232,2020,2266,1719,895,1478
2,90003,2018,4.076667e+05,962,1222.0,96.000000,45.416667,99.933333,314.166667,90090003,...,193,129,17127,2594,2479,2966,3345,2757,1476,1510
3,90004,2018,1.286500e+06,747,1117.0,85.833333,33.833333,99.233333,630.916667,90090004,...,483,463,21971,6379,6385,3842,2854,1539,721,251
4,90005,2018,7.710000e+05,314,446.0,30.666667,30.250000,99.225000,523.000000,90090005,...,431,380,16442,6298,4939,2183,1794,832,293,103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1332,96146,2018,5.807500e+05,249,368.0,62.666667,163.333333,95.691667,486.333333,96196146,...,0,9,454,152,215,31,24,32,0,0
1333,96148,2018,6.440833e+05,155,156.0,14.583333,90.500000,96.950000,413.666667,96196148,...,24,0,210,55,62,6,50,26,0,11
1334,96150,2018,4.536667e+05,2333,3039.0,246.333333,43.750000,97.008333,307.666667,96196150,...,461,24,11536,3645,4315,1666,1221,427,202,60
1335,96155,2018,2.340000e+05,8,NaN,NaN,104.000000,91.333333,145.666667,96196155,...,0,0,0,0,0,0,0,0,0,0


In [9]:
#create avg_sale_to_list groups (split into 5)
ntile_DoM_query = """SELECT *, NTILE(5) OVER (
                            ORDER BY days_on_market ASC) as DoM_group      
                        FROM sales_demo"""
ntile_DoM = ps.sqldf(ntile_DoM_query, locals())


#see group maximums
minmax_DoM_query = """SELECT DoM_group, COUNT(zip_code), MAX(days_on_market)
                        FROM ntile_DoM
                        GROUP BY DoM_group"""
minmax_DoM = ps.sqldf(minmax_DoM_query, locals())
minmax_DoM

,DoM_group,COUNT(zip_code),MAX(days_on_market)
0,1,268,21.416667
1,2,268,34.750000
2,3,267,44.083333
3,4,267,59.416667
4,5,267,2833.000000


In [10]:
#Delete non-numeric and unused columns.
del_col = ['Geo_FIPS', 'Geo_GEOID', 'Geo_NAME', 'Geo_QName', 'Geo_STUSAB', 'Geo_SUMLEV', 
           'Geo_GEOCOMP', 'Geo_FILEID', 'Geo_LOGRECNO', 'Geo_US', 'Geo_REGION', 'Geo_DIVISION', 
          'Geo_STATECE', 'Geo_STATE', 'Geo_COUNTY', 'Geo_COUSUB', 'Geo_PLACE', 'Geo_PLACESE', 
          'Geo_TRACT', 'Geo_BLKGRP', 'Geo_CONCIT', 'Geo_AIANHH', 'Geo_AIANHHFP', 'Geo_AIHHTLI', 
          'Geo_AITSCE', 'Geo_AITS', 'Geo_ANRC', 'Geo_CBSA', 'Geo_CSA', 'Geo_METDIV', 'Geo_MACC', 
          'Geo_MEMI', 'Geo_NECTA', 'Geo_CNECTA', 'Geo_NECTADIV', 'Geo_UA', 'Geo_UACP', 'Geo_CDCURR', 
          'Geo_SLDU', 'Geo_SLDL', 'Geo_VTD', 'Geo_ZCTA3', 'Geo_SUBMCD', 'Geo_SDELM', 'Geo_SDSEC', 
          'Geo_SDUNI', 'Geo_UR', 'Geo_PCI', 'Geo_TAZ', 'Geo_UGA', 'Geo_BTTR', 'Geo_BTBG', 
          'Geo_PUMA5', 'Geo_PUMA1', 'year','Geo_ZCTA5', 'avg_sale_to_list']
for i in del_col:
    del ntile_DoM[i]

### Building the Model

In [11]:
# Set variables to hold test and training data.

# Shuffle data frame.
shuffled = ntile_DoM.sample(frac=1).reset_index(drop=True)

# Remove blanks and NaNs
shuffled.replace('', np.nan)
shuffled = shuffled.dropna(axis=0, how='any')

# Split into data and labels.
labels = shuffled["DoM_group"]
del shuffled["DoM_group"]
zip_codes = shuffled["zip_code"]
del shuffled["zip_code"]
shuffled = shuffled.apply(lambda x: pd.to_numeric(x))

# Create 80% split point.
split = int(len(shuffled)*0.8//1)

# Store in variables.
train_data = shuffled[:split]
train_labels = labels[:split]
test_data = shuffled[split+1:]
test_labels = labels[split+1:]

In [54]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# we want to normalize all of our data if possible
sc = StandardScaler()
X_train = sc.fit_transform(train_data)
X_test = sc.transform(test_data)

#now perform PCA
pca = PCA(n_components=20)
#pca.fit(X_train)

X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

print(pca.explained_variance_ratio_, np.sum(pca.explained_variance_ratio_))
print(pca.singular_values_)

[0.3989909  0.15741318 0.10676879 0.05174286 0.0287918  0.02303178
 0.01975653 0.01775321 0.01589618 0.01516186 0.01150166 0.00995284
 0.00899551 0.00813939 0.00804748 0.00634838 0.00582772 0.00544345
 0.00511428 0.0045773 ] 0.9092550972991045
[152.0707484   95.51789237  78.66587009  54.76327265  40.85061381
  36.53658618  33.8391535   32.07765968  30.35362679  29.64425213
  25.81929685  24.01804962  22.83374097  21.72001257  21.59702877
  19.18207737  18.37864384  17.76239377  17.21696429  16.28803467]


In [ ]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(max_depth=12, random_state=0)
classifier.fit(X_train, train_labels)

# Predicting the Test set results
predicted_labels = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

# Calculate number of incorrect values for each model.
num_wrong = sum(test_labels != predicted_labels)
accuracy = (len(test_labels) - num_wrong) / len(test_labels)
print("Accuracy of our PCA model w/Random Forest : ", accuracy)

In [78]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel

model = XGBClassifier()
model.fit(train_data, train_labels)
# make predictions for test data and evaluate
y_pred = model.predict(test_data)

print(y_pred)

# Calculate number of incorrect values for each model.
num_wrong = sum(test_labels != y_pred)
accuracy = (len(test_labels) - num_wrong) / len(test_labels)
print(accuracy)

# accuracy = accuracy_score(test_data, predictions)
# print("Accuracy: %.2f%%" % (accuracy * 100.0))
# Fit model using each importance as a threshold

thresholds = np.sort(model.feature_importances_)

for thresh in thresholds:
    if(thresh > 0.00001):
        # select features using threshold
        selection = SelectFromModel(model, threshold=thresh, prefit=True)
        select_X_train = selection.transform(train_data)
        # train model
        selection_model = XGBClassifier()
        selection_model.fit(select_X_train, train_labels)
        # eval model
        select_X_test = selection.transform(test_data)
        y_pred = selection_model.predict(select_X_test)
        #predictions = [round(value) for value in y_pred]
        accuracy = accuracy_score(test_labels, y_pred)
        print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))

[2 1 2 1 1 4 2 1 1 1 2 3 4 3 3 3 4 4 4 1 1 1 1 2 1 4 5 4 4 1 4 4 1 4 3 4 2
 4 5 1 5 5 3 3 1 1 3 4 1 2 2 4 4 1 1 3 5 2 1 5 2 3 2 1 4 5 2 5 1 3 1 3 3 5
 3 3 2 1 1 1 3 3 3 2 3 3]
0.9767441860465116
Thresh=0.001, n=15, Accuracy: 97.67%
Thresh=0.001, n=14, Accuracy: 97.67%
Thresh=0.001, n=13, Accuracy: 97.67%
Thresh=0.002, n=12, Accuracy: 97.67%
Thresh=0.002, n=11, Accuracy: 97.67%
Thresh=0.002, n=10, Accuracy: 97.67%
Thresh=0.002, n=9, Accuracy: 97.67%
Thresh=0.002, n=8, Accuracy: 97.67%
Thresh=0.002, n=7, Accuracy: 97.67%
Thresh=0.002, n=6, Accuracy: 97.67%
Thresh=0.003, n=5, Accuracy: 97.67%
Thresh=0.003, n=4, Accuracy: 97.67%
Thresh=0.003, n=3, Accuracy: 97.67%
Thresh=0.006, n=2, Accuracy: 97.67%
Thresh=0.968, n=1, Accuracy: 97.67%
